In [33]:
import os
import string
import re
import math
import statistics
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)
np.random.seed(1)
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle

#os.chdir('C:/Users/torna/Documents/StatApp/StatApp')
#os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')
#os.chdir('/home/aqlt/Documents/Ensae/StatApp')
os.chdir('C:/Users/Kim Antunez/Documents/Projets_autres/StatApp')

nom_dossier = "100k"

In [34]:
# Penser à changer selon taille
with open("data/corpus_trie%s.file" %nom_dossier, "rb") as f:
    corpus = pickle.load(f) 
ens_tweets = [phrase.split() for phrase in corpus]
phrases = ens_tweets
print(len(phrases))

99718


# Lancement du modèle
Paramètres fixés : la dimension, le nombre de mots tirés dans le *negative sampling* et la proba utilisé, l'affichage du graphique

In [49]:
if nom_dossier == "ens":
    dim = 50
else:
    dim = 20
plot = True

Paramètres à modifier :

In [62]:
epoch = 100
learning_rate = 0.02
window = 5
numero_simulation = 12
seeds = [1, 5, 10, 15, 20, 25]
#seeds = [1]

In [63]:
# phrases = les phrases du corpus
# size = dimension du vecteur
# window = fenetre
# negative = nb de neg samples
# alpha = learning rate
# seed 
# min_count = fréquence min des mots qui nous intéresse
# workers = nb de cores sur l'ordi
# iter = nb epoch

for seed in seeds:

    start = time.time()

    from gensim.models import word2vec
    model = word2vec.Word2Vec(phrases, size=dim, window=window,negative=5,alpha=learning_rate,seed=seed,
                      min_count=0, workers=4, iter=epoch)
    # Kim : j'ai mis min_count = 0 et non 5 car sinon on obtenait pas le même voc que dans NOTRE modele. a creuser

    end = time.time()
    # Affichage temps de tournage du modèle
    print("seed", seed, " : ", round((end - start)/60, 2))

    # Sauvegarde 
    chemin = "data/" + nom_dossier + "/gensim/Simulation_" + str(numero_simulation) +"_seed"+ str(seed)

    if not os.path.isdir(chemin):
        os.mkdir(chemin)

    model.save(chemin + "/word2vec.model")

seed 1  :  1.25
seed 5  :  1.11
seed 10  :  1.02
seed 15  :  1.02
seed 20  :  1.03
seed 25  :  1.1


#  Quelques fonctions intégrées dans gensim (non utilisées)

In [9]:
model.corpus_count

99718

In [10]:
vocab = model.wv.vocab
list(vocab)[:5]

['il', 'mérite', 'd', 'être', 'bloquer']

In [11]:
model.wv['wesh']

array([-0.8029589 ,  0.5978353 , -0.49241248, -0.5030643 , -0.03345106,
       -0.0076034 ,  0.26484555,  0.8762143 , -0.4996161 ,  0.08204883,
       -0.08663379,  0.75073415,  0.6436939 , -0.14495811,  0.647974  ,
        0.6855672 , -0.269488  ,  0.2696291 ,  0.49807718, -0.59831595],
      dtype=float32)

In [12]:
model.similarity("homme","femme")

C:\Users\Kim Antunez\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.8841527